In [21]:
import argparse
import torch
import torch.nn as nn
import numpy as np
import os
from MLP import MLP 
import pickle
from data_loader import load_test_dataset, data_loader
from torch.autograd import Variable 
import math
import time
import pandas as pd

This code is used to load dataset

In [23]:
file_path = r"/home/sj/Deep-RRT-Star-Implementation/output.csv"
dtypes = {col: 'float32' for col in pd.read_csv(file_path, nrows=1).columns}
example = pd.read_csv(file_path, dtype=dtypes)
index1 = 0
index3 = 1
prev_image_number = 0
environment_list = [0]
print(len(example))
while index1 < len(example) :
    row = example.iloc[index1]
    index_start = row['index_start']
    index_goal = row['index_goal']
    image_number = row['image_number']
    index1 = index_goal + 1
    print(index_goal)
    index1 = int(index1)
    if (image_number == prev_image_number):
        index3 = index3 + 1
    else :
        environment_list.append(prev_image_number)
        index3 = 1
        prev_image_number = image_number
        index3 = index3 + 1

dataset , targets = data_loader(environment_list,101,201)


317938
5.0
23.0
59.0
86.0
125.0
137.0
158.0
165.0
185.0
192.0
205.0
217.0
239.0
273.0
285.0
301.0
318.0
325.0
330.0
340.0
352.0
363.0
382.0
432.0
437.0
460.0
487.0
492.0
517.0
523.0
555.0
560.0
578.0
596.0
626.0
631.0
635.0
665.0
676.0
686.0
712.0
729.0
753.0
787.0
796.0
826.0
849.0
853.0
872.0
877.0
904.0
914.0
929.0
947.0
962.0
976.0
985.0
1004.0
1006.0
1028.0
1044.0
1048.0
1062.0
1081.0
1090.0
1118.0
1150.0
1180.0
1190.0
1194.0
1203.0
1207.0
1239.0
1246.0
1257.0
1293.0
1306.0
1321.0
1324.0
1328.0
1375.0
1403.0
1411.0
1449.0
1483.0
1492.0
1504.0
1541.0
1558.0
1575.0
1590.0
1602.0
1614.0
1637.0
1667.0
1674.0
1689.0
1709.0
1730.0
1734.0
1760.0
1781.0
1788.0
1797.0
1808.0
1827.0
1841.0
1888.0
1904.0
1916.0
1918.0
1921.0
1933.0
1949.0
1956.0
1958.0
1982.0
1988.0
1994.0
2022.0
2039.0
2069.0
2075.0
2087.0
2104.0
2121.0
2125.0
2131.0
2145.0
2160.0
2180.0
2186.0
2193.0
2212.0
2244.0
2262.0
2290.0
2301.0
2325.0
2353.0
2368.0
2376.0
2384.0
2401.0
2420.0
2430.0
2445.0
2471.0
2493.0
2523.0
2534.

RuntimeError: Attempting to deserialize object on a CUDA device but torch.cuda.is_available() is False. If you are running on a CPU-only machine, please use torch.load with map_location=torch.device('cpu') to map your storages to the CPU.

In [22]:
def to_var(x, volatile=False):
	if torch.cuda.is_available():
		x = x.cuda()
	return Variable(x, volatile=volatile)

def get_input(i,dataset,targets,seq,bs):
	bi=np.zeros((bs,18),dtype=np.float32)
	bt=np.zeros((bs,2),dtype=np.float32)
	k=0	
	for b in range(i,i+bs):
		bi[k]=dataset[seq[i]].flatten()
		bt[k]=targets[seq[i]].flatten()
		k=k+1
	return torch.from_numpy(bi),torch.from_numpy(bt)

# Load trained model for path generation
mlp = MLP(32, 2) # simple @D
mlp.load_state_dict(torch.load('models/mlp_100_4000_PReLU_ae_dd150.pkl'))

if torch.cuda.is_available():
	mlp.cuda()

#load test dataset
obc,obstacles, paths, path_lengths= load_test_dataset() 

FileNotFoundError: [Errno 2] No such file or directory: 'models/mlp_100_4000_PReLU_ae_dd150.pkl'

In [ ]:
def replan_path(p,g,idx,obs):
	step=0
	path=[]
	path.append(p[0])
	for i in range(1,len(p)-1):
		if not isInObstacle(p[i],idx):
			path.append(p[i])
	path.append(g)			
	new_path=[]
	for i in range(0,len(path)-1):
		target_reached=False

	 
		st=path[i]
		gl=path[i+1]
		steer=steerTo(st, gl, idx)
		if steer==1:
			new_path.append(st)
			new_path.append(gl)
		else:
			itr=0
			pA=[]
			pA.append(st)
			pB=[]
			pB.append(gl)
			target_reached=0
			tree=0
			while target_reached==0 and itr<50 :
				itr=itr+1
				ip1=torch.cat((obs,st,gl))
				ip1=to_var(ip1)
				st=mlp(ip1)
				st=st.data.cpu()
				pA.append(st)
				target_reached = goal_found(point)
			if target_reached==0:
				return 0
			else:
				for p1 in range(0,len(pA)):
					new_path.append(pA[p1])
	return new_path

In [ ]:
import cv2
from data_loader import Autoencoder
from MLP import MLP
from torchvision import datasets, transforms
import torch

# Encoding the image
autoencoder = Autoencoder()
Q = autoencoder.encoder
Q.load_state_dict(torch.load(r"/home/sj/Deep-RRT-Star-Implementation/weights/cae_encoder2.pkl"))
if torch.cuda.is_available():
    Q = Q.to(torch.device("cuda"))
    
# Load trained model for path generation
mlp = MLP(32, 2)
mlp.load_state_dict(torch.load('models/mlp_100_4000_PReLU_ae_dd150.pkl'))

if torch.cuda.is_available():
    mlp.cuda()

# Load test dataset
image_path = r"/home/sj/Deep-RRT-Star-Implementation/images/180.jpg"
path = []

image = cv2.imread(image_path)

encoded_w_m=np.zeros((1,28),dtype=np.float32)
to_tensor = transforms.ToTensor()
torch_img = to_tensor(image)
if torch.cuda.is_available():
    torch_img = torch_img.to(torch.device("cuda")) 

output=Q(torch_img)
output = output.squeeze()
output=output.data.cpu()
output = output.numpy()
encoded_w_m[0] = output

start_x = 
start_y =
goal_x = 
goal_y =
start = np.array([start_x, start_y])
goal = np.array([goal_x, goal_y])
path.append(start)


start1=torch.from_numpy(start)
goal1=torch.from_numpy(goal)
obs = torch.from_numpy(encoded_w_m)

# Implement the Network to generate the path
while not goalFound(current):
    start1 = torch.cat((obs,start1,goal1))
    start1 = to_var(start1)
    current = mlp(start1)
    current = current.data.cpu()
    path.append(current)
    start1 = current.numpy()
    if goalFound(start1):
        path.append(goal1)
        break

In [24]:
import numpy as np
import matplotlib.pyplot as plt
import random
from matplotlib.pyplot import rcParams
import csv
import cv2

class treeNode():
    def __init__(self, locationX, locationY):
        self.locationX = locationX                #X Location
        self.locationY = locationY                #Y Location  
        self.children = []                        #children list   
        self.parent = None                        #parent node reference 
        
#RRT Star Algorithm class
class RRTStarAlgorithm():
    def __init__(self, start, goal, numIterations, grid, stepSize, row_number, image_number):
        self.randomTree = treeNode(start[0], start[1])          #The RRT (root position) (has 0 cost)
        self.goal = treeNode(goal[0], goal[1])                  #goal position (initialize to a high cost)
        self.nearestNode = None                                 #nearest node            
        self.iterations = min(numIterations, 700)               #number of iterations to run
        self.grid = grid                                        #the map
        self.rho = stepSize                                     #length of each branch   
        self.nearestDist = 10000                                #distance to nearest node (initialize with large)
        self.numWaypoints = 0                                   #number of waypoints
        self.Waypoints = []                                     #the waypoints
        self.searchRadius = self.rho*2                          #the radius to search for finding neighbouring vertices 
        self.neighbouringNodes = []                             #neighbouring nodes  
        self.goalArray = np.array([goal[0],goal[1]])            #goal as an array
        self.goalCosts = [10000]                                #the costs to the goal (ignore first value)
        self.row_number = row_number
        self.image_number = image_number
        
        
        
            
    #add the node to the nearest node, and add goal if necessary (TODO--------)     
    def addChild(self, treeNode):
        if (treeNode.locationX == self.goal.locationX):
            #append goal to nearestNode's children
            self.nearestNode.children.append(self.goal)
            #and set goal's parent to nearestNode
            self.goal.parent = self.nearestNode
        else:    
            #append this node to nearestNode's children
            self.nearestNode.children.append(treeNode)
            #set the parent to nearestNode
            treeNode.parent = self.nearestNode
        
    #sample random point within grid limits (DONE)
    def sampleAPoint(self):
        x = random.randint(1, grid.shape[1])
        y = random.randint(1, grid.shape[0])
        point = np.array([x, y])
        return point
    
    #steer a distance stepSize from start location to end location (keep in mind the grid limits) (DONE)
    def steerToPoint(self, locationStart, locationEnd):
        offset = self.rho*self.unitVector(locationStart, locationEnd)
        point = np.array([locationStart.locationX + offset[0], locationStart.locationY + offset[1]])
        if point[0] >= grid.shape[1]:
            point[0] = grid.shape[1]-1
        if point[1] >= grid.shape[0]:
            point[1] = grid.shape[0]-1
        return point
    
    #check if obstacle lies between the start and end point of the edge (DONE)
    def isInObstacle(self, locationStart, locationEnd):
        u_hat = self.unitVector(locationStart, locationEnd)
        testPoint = np.array([0.0, 0.0])
        dist = self.distance(locationStart, locationEnd)
        # Check if dist is NaN
        if np.isnan(dist):
            return False
        for i in range(int(dist)):
            testPoint[0] = min(grid.shape[1]-1, locationStart.locationX + i*u_hat[0])
            testPoint[1] = min(grid.shape[0]-1,locationStart.locationY + i*u_hat[1])
            if self.grid[round(testPoint[1]),round(testPoint[0])] == 0:
                return True
        return False

    #find the unit vector between 2 locations (DONE)
    def unitVector(self, locationStart, locationEnd):
        v = np.array([locationEnd[0] - locationStart.locationX, locationEnd[1] - locationStart.locationY])
        u_hat = v/np.linalg.norm(v)
        return u_hat
    
    #find the nearest node from a given (unconnected) point (Euclidean distance) (TODO--------)  
    def findNearest(self, root, point):
        if not root:
            return
        #find distance between root and point use distance method,
        distance_to_nearest = self.distance(root, point)
        #if it's lower than or equal to nearestDist then
        if distance_to_nearest <= self.nearestDist:
            #update nearestNode to root
            self.nearestNode = root
            #update nearestDist to the distance from line 84
            self.nearestDist = distance_to_nearest
        #recursive call
        for child in root.children:
            self.findNearest(child, point)
            
    #find neighbouring nodes (TODO--------)         
    def findNeighbouringNodes(self,root,point):
        if not root:
            return
        #find distance between root and point (dist)
        distance_root_point = self.distance(root, point)
        #add root to neighbouringNodes if dist is less than or equal to searchRadius
        if distance_root_point <= self.searchRadius:
            self.neighbouringNodes.append(root)
        #recursive call
        for child in root.children:
            self.findNeighbouringNodes(child, point)        

    #find euclidean distance between a node and an XY point (DONE)
    def distance(self, node1, point):
        dist = np.sqrt((node1.locationX - point[0])**2 + (node1.locationY - point[1])**2)         
        return dist
    
    #check if the goal is within stepsize (rho) distance from point, return true if so otherwise false (TODO--------)
    def goalFound(self,point):
        if self.distance(self.goal, point) <= self.rho:
            return True
        return False
            
    #reset: set nearestNode to None, nearestDistance to 10K and neighbouringNodes to empty array
    def resetNearestValues(self):
        self.nearestNode = None
        self.nearestDist = 10000
        self.neighbouringNodes = []
    
    #trace the path from goal to start, since have to reset if called many times, do this iteratively (TODO--------)
    def retracePath(self):
        self.numWaypoints = 0
        self.Waypoints = []
        goalCost = 0
        goal = self.goal
        while goal.locationX != self.randomTree.locationX:
            self.numWaypoints += 1
            # goal = np.array([goal.locationX, goal.locationY])
            self.Waypoints.insert(0, np.array([goal.locationX, goal.locationY]))
            goalCost += self.distance(goal.parent, np.array([goal.locationX, goal.locationY]))
            goal = goal.parent  #set the node to it's parent
        self.goalCosts.append(goalCost)    
        output_file = r"C:\Users\sachi\Planning\Deep-RRT-Star-Implementation\output2.csv"

        with open(output_file, 'a', newline='') as file:
            writer = csv.writer(file)
            if file.tell() == 0: # Checks if the file is empty
                writer.writerow(['current_x', 'current_y', 'index_start', 'index_goal', 'image_number'])
            index_start = self.row_number 
            index_goal = index_start + len(rrtStar.Waypoints) - 2
            for i in range(len(rrtStar.Waypoints)-1):
                current_x, current_y = rrtStar.Waypoints[i][0], rrtStar.Waypoints[i][1]
                if i == 0:
                    writer.writerow([start_x, start_y, index_start, index_goal, image_number])
                elif i == len(rrtStar.Waypoints)-2:
                    writer.writerow([goal_x, goal_y])
                else:
                    writer.writerow([current_x, current_y])
                self.row_number += 1
        return self.row_number

    #find unique path length from root of a node (cost) (DONE)
    def findPathDistance(self, node):
        costFromRoot = 0
        currentNode = node 
        while currentNode and currentNode.parent and currentNode.locationX != self.randomTree.locationX:
                parent_location = np.array([currentNode.parent.locationX, currentNode.parent.locationY])
                costFromRoot += self.distance(currentNode, parent_location)
                currentNode = currentNode.parent   
                if currentNode is None or currentNode.parent is None:
                    return 0
        return costFromRoot
